In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [ ]:
#прочитаь 1 столбец названиями
excel = pd.read_excel('./DILIst Supplementary Table.xlsx', sheet_name='DILIst')
names = excel['CompoundName'].tolist()
print(names)

In [94]:
def readDict(filename, sep):
    with open(filename, "r") as f:
        dict = {}
        for line in f:
            values = line.replace(",","").strip().split(sep)
            dict[values[0].replace("'","")] = int(values[1])
        return(dict)


In [95]:
ch_dict = readDict('chemspider.txt', ':')

In [105]:
def get_logd(id, tag, d):
  url_tag = 'http://www.chemspider.com/Chemical-Structure.{}.html'.format(id);
  page = requests.get(url_tag)
  soup = BeautifulSoup(page.text, "html.parser")
  lst_props = []
  lst_vals = []
  lst_props = soup.findAll('td', class_= 'prop_title')
  div = soup.find('div',  {'id': 'suppInfoTab'})
  lst_div = []
  if div:
    lst_div = div.findAll('td')
  d['toxicity'] = ''
  if lst_div:
    for l in lst_div:
      if "LD50" in l.getText():
        d['toxicity'] = l.getText()
  for p in lst_props:
    if "ACD/LogD" in p.get_text():
      key = p.get_text().strip()
      val = p.find_next_sibling('td').get_text().strip()
      d[key] = val

In [86]:
d={}

In [ ]:
get_logd(2157, 'aspirin',d)

In [ ]:
data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/5479/xml")
data
html = BeautifulSoup(data.content, "xml")
solubility_tag = html.find(name='TOCHeading', string='Solubility')
siblings = solubility_tag.find_next_siblings('Information')
for sib in siblings:
  if sib.find(name='Number'):
    solubility = '{} {}'.format(
    sib.find(name='Number').string,
    sib.find(name='Unit').string
    )
    print(solubility)


In [ ]:
molecules = []

In [98]:
def get_bf_data(tag, temp):
  temp['name'] = tag
  url_tag = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{}/xml'.format(tag);
  data = requests.get(url_tag)
  html = BeautifulSoup(data.content, "xml")
  #print(html)
  try:
    smile_tag = html.find(name="PC-Urn_label", string="SMILES")
    smile_parents = smile_tag.find_parent("PC-InfoData")
    smile = smile_parents.find('PC-InfoData_value_sval').string
    temp['smile'] = smile
  except:
    temp['smile'] = ' '
    print("Error in smile", tag)
  try:
    log_tag = html.find(name="PC-Urn_label", string="Log P")
    log_parents = log_tag.find_parent("PC-InfoData")
    log_p = log_parents.find('PC-InfoData_value_fval').string
    temp['log_p'] = log_p
  except:
    temp['log_p'] = ' '
    print("Error in log_p", tag)
  try:
    id_tag = html.find(name="PC-CompoundType_id_cid").string
    get_pug_rest(id_tag, temp)
  except:
    print("Error in toxicity & solubility", tag)
  return temp

In [ ]:
for key, value in ch_dict.items():
  temp = {}
  get_bf_data(key, temp)
  get_logd(value, key, temp)
  print(temp)
  molecules.append(temp)

In [ ]:
get_bf_data('tinidazole')

In [97]:
def get_pug_rest(id, d):
    url_tag = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{}/xml'.format(id)
    data = requests.get(url_tag)
    html = BeautifulSoup(data.content, "xml")
    try:
      solubility_tag = html.find(name='TOCHeading', string='Solubility')
      siblings = solubility_tag.find_next_siblings('Information')
      solubility = ' '
      for sib in siblings:
        if sib.find(name='Number'):
          solubility = '{} {}'.format(
          sib.find(name='Number').string,
          sib.find(name='Unit').string
          )
      d['solubility'] = solubility
    except:
      print("Error in", id)
      d['solubility'] = ' '
    try:
      toxicity_tag = html.find(name='TOCHeading', string='Toxicity Summary')
      d['toxicity_sum'] = toxicity_tag.find_next_sibling('Information').find(name='String').string
    except:
      d['toxicity_sum'] = ' '
      print("Error in", id)

In [ ]:
data = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/667490/xml')
html = BeautifulSoup(data.content, "xml")
html

In [ ]:
for i in range(len(names)): 
  print(names[i])
  molecules.append(get_bf_data(names[i].lower()))

In [99]:
molecules=[]

In [ ]:
import csv
with open('test_molecules_with_logd','w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(('Name','Smile','Log P', 'Log D (ph 5.5)', 'Log D (ph 7.4)','Solubility', 'Toxicity'))
        for mol in molecules:
            try:
              writer.writerow((mol['name'], mol['smile'], mol['log_p'], 
                               mol['ACD/LogD (pH 5.5):'], mol['ACD/LogD (pH 7.4):'], 
                               mol['solubility'], mol['toxicity']))
            except: 
              print(mol)

In [ ]:
!pip install chemspipy

In [4]:
from chemspipy import ChemSpider

In [5]:
cs = ChemSpider('pm3w5rO3yTm9qTDSnTBccPSkbMHx4HWp')

In [6]:
chemspider_list = {}
absent_list=[]

In [7]:
absent_list=[]

In [ ]:
for n in names:
    print(n) 
    tmp = cs.search(n)
    if len(tmp)> 0:
      chemspider_list[n] = tmp[0].record_id
      print(n)
    else:
      absent_list.append(n)

In [ ]:
chemspider_list

{}

In [ ]:
len(chemspider_list)

0

In [ ]:
len(absent_list)

1279